In [12]:
import rioxarray as rxr
import os
from rasterio.plot import show
import PIL
from PIL import Image
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import rasterio
import geopandas as gpd
import rasterio.features
from shapely.geometry import shape
import pandas as pd
import time
import tifffile as tiff
import random

In [2]:
# retreive some infos on fully contained segments
contained_file_50 = 'data/contained_segments_50.txt'
contained_file_0001 = 'data/contained_segments_0001.txt'
contained_file_100 = 'data/contained_segments_100.txt'

- Satellite raster data (Sentinel 2)
	- 10 spectral bands, one per file (s2_2020_B02.tif, s2_2020_B03.tif, etc...)
	- Each file s2_2020_BXX.tif: contains an array of size (73, 4513, 5183)=(nb. dates, height, width).

In [3]:
def load_data_from_file(contained_file):
    # load the data
    with open(contained_file) as f:
        lines = f.readlines()
        # reteive line infos
        segment_id = []
        polygon_id = []
        class_id = []
        pixels = []
        for line in lines:
            # retrieve segment id
            segment_id.append(line.split(',')[0].split(':')[1].strip())
            # retrieve polygon id
            polygon_id.append(line.split(',')[1].split(':')[1].strip())
            # retrieve class id
            class_id.append(line.split(',')[2].split(':')[1].strip())
            # retrieve pixels
            pixel_list_str = line.split(':')[4]
            pixel_list_str = pixel_list_str[3:-3]
            pixel_list = list(pixel_list_str.split('], ['))
            pixel_list = [pixel.split(', ') for pixel in pixel_list]
            pixel_list = [[int(pixel[0]), int(pixel[1])] for pixel in pixel_list]
            pixels.append(pixel_list)
        
        # create a dataframe
        df = pd.DataFrame()
        df['segment_id'] = segment_id
        df['polygon_id'] = polygon_id
        df['class_id'] = class_id
        df['pixels'] = pixels
        
        return df

In [4]:
dataframe_segment = load_data_from_file(contained_file_50)

In [5]:
# show mean lenth of segments' pixels
mean_lenth = []
for pixels in dataframe_segment['pixels']:
    mean_lenth.append(len(pixels))
print("mean lenth of segments' pixels")
print(np.mean(mean_lenth))

# show mean number of segments per polygon
print("mean number of segments per polygon")
print(len(dataframe_segment['segment_id'].unique())/len(dataframe_segment['polygon_id'].unique()))

# show number of polygons
print("number of polygons")
print(len(dataframe_segment['polygon_id'].unique()))

# show number of segments
print("number of segments")
print(len(dataframe_segment['segment_id'].unique()))

# show total number of pixels
print("total number of pixels")
print(len(dataframe_segment['pixels'].sum()))

mean lenth of segments' pixels
16.178727841501566
mean number of segments per polygon
6.382157123834887
number of polygons
751
number of segments
4793
total number of pixels
77577


In [6]:
# create a new dataframe for output
dataframe_segment_output = load_data_from_file(contained_file_50)
# change each pixel of each segment to an empty list
for index_output, row_output in dataframe_segment_output.iterrows():
    for i_output, pixel_output in enumerate(row_output['pixels']):
        row_output['pixels'][i_output] = []

In [7]:
folder = "D:/General/ExaplAInability_Data/transfer_6060512_files_e989f8bb/normalized_gathered_output"
list_bands = ['02']#, '03', '04', '05', '06', '07', '08', '8A', '11', '12']
list_band_files = [rf'{folder}/normalized_s2_2020_B{band}.tif' for band in list_bands]

In [16]:
# open the tif file
band = '02'
file_path = os.path.join(folder, f'normalized_s2_2020_B{band}.tif')
random_5_pixels = [random.randint(0, 10) for i in range(100)]

with rasterio.open(file_path) as src:
    
    '''    # measure time taken to read values at coordinates for 5 pixels
        start_time = time.time()
        data = src.read(1)
        for pixel in random_5_pixels:
            x, y = pixel, pixel
            value = data[x, y]
            print(value)
        print("Time taken to read values at coordinates for 5 pixels:", time.time() - start_time)
    '''
    
    # measure time taken to read all values for 5 pixels
    start_time = time.time()
    for pixel in random_5_pixels:
        x, y = pixel, pixel
        value = src.read(1)[x][y]
        print(value)
    print("Time taken to read all values for 5 pixels:", time.time() - start_time)


0.9109698
0.7615262
0.9109698
0.7615262
0.9109698
0.83147854
0.8378378
0.50397456
0.7376788
0.83147854
0.55325913
0.7615262
0.7615262
0.8378378
0.68203497
0.50397456
0.7615262
0.8155803
0.83147854
0.55325913
0.74721783
0.5405405
0.8378378
0.5405405
0.9109698
0.55325913
0.8155803
0.68203497
0.5405405
0.7615262
0.74721783
0.7376788
0.7376788
0.8155803
0.8378378
0.55325913
0.50397456
0.8378378
0.50397456
0.8155803
0.8378378
0.8155803
0.7376788
0.74721783
0.8155803
0.8378378
0.9109698
0.5405405
0.83147854
0.8378378
0.50397456
0.5405405
0.83147854
0.5405405
0.8378378
0.7615262
0.8378378
0.5405405
0.55325913
0.8155803
0.55325913
0.8155803
0.50397456
0.9109698
0.5405405
0.83147854
0.7376788
0.68203497
0.55325913
0.8378378
0.74721783
0.8378378
0.74721783
0.83147854
0.9109698
0.7376788
0.7376788
0.8378378
0.7376788
0.55325913
0.68203497
0.7376788
0.68203497
0.9109698
0.9109698
0.50397456
0.83147854
0.5405405
0.83147854
0.7615262
0.7376788
0.5405405
0.50397456
0.7615262
0.55325913
0.8155803
0.68

In [17]:
start = 1 # always 1
acquisition = 3 # 73
print(f"folder: {folder}")
print(f"list_bands: {list_bands}")
print(f"start: {start}")

# iterate through each band file
for band in list_bands:
    file_path = os.path.join(folder, f'normalized_s2_2020_B{band}.tif')
    print(f"file_path: {file_path}")
    
    with rasterio.open(file_path) as src:
        # for each band file, iterate through all acquisition
        for period in range(start, acquisition+1):
            # for each acquisition, iterate through all segments
            print(f"period: {period}/{acquisition}")
            
            for index, row in dataframe_segment.iterrows():
                # for all segments, iterate through all pixels and add the
                print(index)
                for i, pixel in enumerate(row['pixels']):
                    x, y = pixel
                    value = src.read(period)[x,y]
                    # add the pixel value in the output dataframe
                    dataframe_segment_output.at[index, 'pixels'][i].append(value)

folder: D:/General/ExaplAInability_Data/transfer_6060512_files_e989f8bb/normalized_gathered_output
list_bands: ['02']
start: 1
file_path: D:/General/ExaplAInability_Data/transfer_6060512_files_e989f8bb/normalized_gathered_output\normalized_s2_2020_B02.tif
period: 1/3
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17


KeyboardInterrupt: 

In [18]:
print(dataframe_segment_output)

     segment_id polygon_id class_id  \
0        367854        0.0      3.0   
1        367855        0.0      3.0   
2        367856        0.0      3.0   
3        369130        0.0      3.0   
4        363215        1.0      1.0   
...         ...        ...      ...   
4790     370396      997.0      4.0   
4791     370397      997.0      4.0   
4792     370398      997.0      4.0   
4793     371681      997.0      4.0   
4794     371682      997.0      4.0   

                                                 pixels  
0     [[0.6406995], [0.6581876], [0.45786965], [0.37...  
1     [[0.47058824], [0.49602544], [0.48330683], [0....  
2     [[0.43561208], [0.39427662], [0.3990461], [0.4...  
3     [[0.7837838], [0.6709062], [0.6788553], [0.818...  
4     [[0.5643879], [0.5755167], [0.50874406], [0.54...  
...                                                 ...  
4790  [[], [], [], [], [], [], [], [], [], [], [], [...  
4791  [[], [], [], [], [], [], [], [], [], [], [], [...  
4792  [[]